<a href="https://colab.research.google.com/github/biemt-pesc-coppe-ufrj/exemplosXexeo/blob/master/bimt_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Escolhe alguns textos do Gutemberg (NLTK)

In [ ]:
import nltk
nltk.download('gutenberg')
docs=[0]*6  # Vetor de Documentos

# Escolhe 6 documentos entre os fornecidos
docs[0]=nltk.corpus.gutenberg.words('austen-sense.txt')
docs[1]=nltk.corpus.gutenberg.words('shakespeare-caesar.txt')
docs[2]=nltk.corpus.gutenberg.words('shakespeare-hamlet.txt')
docs[3]=nltk.corpus.gutenberg.words('shakespeare-macbeth.txt')
docs[4]=nltk.corpus.gutenberg.words('melville-moby_dick.txt')
docs[5]=nltk.corpus.gutenberg.words('carroll-alice.txt')


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [ ]:
print(docs[0][:100])


['[', 'Sense', 'and', 'Sensibility', 'by', 'Jane', 'Austen', '1811', ']', 'CHAPTER', '1', 'The', 'family', 'of', 'Dashwood', 'had', 'long', 'been', 'settled', 'in', 'Sussex', '.', 'Their', 'estate', 'was', 'large', ',', 'and', 'their', 'residence', 'was', 'at', 'Norland', 'Park', ',', 'in', 'the', 'centre', 'of', 'their', 'property', ',', 'where', ',', 'for', 'many', 'generations', ',', 'they', 'had', 'lived', 'in', 'so', 'respectable', 'a', 'manner', 'as', 'to', 'engage', 'the', 'general', 'good', 'opinion', 'of', 'their', 'surrounding', 'acquaintance', '.', 'The', 'late', 'owner', 'of', 'this', 'estate', 'was', 'a', 'single', 'man', ',', 'who', 'lived', 'to', 'a', 'very', 'advanced', 'age', ',', 'and', 'who', 'for', 'many', 'years', 'of', 'his', 'life', ',', 'had', 'a', 'constant', 'companion']


# Cria um modelo de unigramas

1. Usando Porter Stemmer para buscar o radical
1. Pegando só palavras com mais de 2 letras e começando por letra (tira os números)


In [ ]:
from nltk.stem import PorterStemmer 

ps = PorterStemmer()

def gen_unigram_model(doc):
  tokens = {}
  total = 0
  for token in doc:
    tokst = ps.stem(token)
    if len(tokst)>2 and ("a"<=tokst[0]<="z"):
      tokens[tokst] = tokens.get(tokst,0)+1
      total += 1
  for key in tokens:
    tokens[key] /= total
  return tokens

In [ ]:
test_model = gen_unigram_model(docs[5])

i = 0
for key in test_model:
  print(f"{key:10} ==> {test_model[key]:10f} ")
  i += 1
  if i > 100:
    break

alic       ==>   0.019314 
adventur   ==>   0.000340 
wonderland ==>   0.000146 
lewi       ==>   0.000049 
carrol     ==>   0.000049 
chapter    ==>   0.000582 
down       ==>   0.004950 
the        ==>   0.079682 
rabbit     ==>   0.002523 
hole       ==>   0.000243 
begin      ==>   0.001504 
get        ==>   0.003300 
veri       ==>   0.006988 
tire       ==>   0.000340 
sit        ==>   0.000922 
her        ==>   0.012229 
sister     ==>   0.000534 
bank       ==>   0.000194 
and        ==>   0.042316 
have       ==>   0.004367 
noth       ==>   0.001650 
onc        ==>   0.001650 
twice      ==>   0.000243 
she        ==>   0.026836 
had        ==>   0.008638 
peep       ==>   0.000194 
into       ==>   0.003251 
book       ==>   0.000631 
read       ==>   0.000679 
but        ==>   0.008250 
pictur     ==>   0.000291 
convers    ==>   0.000534 
what       ==>   0.006842 
use        ==>   0.001698 
thought    ==>   0.003882 
without    ==>   0.001262 
consid     ==>   0.000485 
o

In [ ]:
print(list(test_model.items()))

[('alic', 0.019313825399136217), ('adventur', 0.00033969039646721986), ('wonderland', 0.00014558159848595138), ('lewi', 4.852719949531713e-05), ('carrol', 4.852719949531713e-05), ('chapter', 0.0005823263939438055), ('down', 0.004949774348522347), ('the', 0.07968166157131072), ('rabbit', 0.0025234143737564905), ('hole', 0.00024263599747658562), ('begin', 0.0015043431843548309), ('get', 0.0032998495656815646), ('veri', 0.006987916727325666), ('tire', 0.00033969039646721986), ('sit', 0.0009220167904110254), ('her', 0.012228854272819915), ('sister', 0.0005337991944484884), ('bank', 0.0001941087979812685), ('and', 0.04231571795991653), ('have', 0.004367447954578541), ('noth', 0.0016499247828407823), ('onc', 0.0016499247828407823), ('twice', 0.00024263599747658562), ('she', 0.02683554132091037), ('had', 0.008637841510166448), ('peep', 0.0001941087979812685), ('into', 0.003251322366186247), ('book', 0.0006308535934391226), ('read', 0.0006793807929344397), ('but', 0.008249623914203912), ('pict

# Cria o modelo invertido para gerar documentos

Isso não é usado na recuperação, foi feito só para teste


In [ ]:
def invert_model(model):
  ans = []
  for key, value in model.items():
    temp = [key,value]
    ans.append(temp)
  ans.sort()
  total = 0
  for item in ans:
    item[1] += total
    total = item[1]
  return(ans)

import random

def next_emission(imodel):
  luck = random.random()
  last_item = imodel[0]
  for item in imodel:
    if luck < item[1]:
      return item[0]

def generate(how_many,model):
  imodel = invert_model(model)
  sentence = []
  for i in range(how_many):
    sentence.append(next_emission(imodel))
  return sentence





In [ ]:
print(invert_model(test_model))


[['abid', 4.852719949531713e-05], ['abl', 9.705439899063426e-05], ['about', 0.004658611151550444], ['abov', 0.0048041927500363955], ['absenc', 0.004852719949531713], ['absurd', 0.004949774348522347], ['accept', 0.004998301548017664], ['accid', 0.005095355947008298], ['accident', 0.005143883146503616], ['account', 0.005289464744989567], ['accus', 0.005337991944484885], ['accustom', 0.005386519143980202], ['ach', 0.0054350463434755195], ['across', 0.005677682340952105], ['act', 0.005726209540447422], ['actual', 0.00577473673994274], ['ada', 0.005823263939438057], ['address', 0.005968845537924009], ['adjourn', 0.006017372737419326], ['adopt', 0.0060658999369146436], ['advanc', 0.006211481535400595], ['advantag', 0.006357063133886546], ['adventur', 0.006696753530353766], ['advic', 0.0067938079293444], ['advis', 0.006939389527830351], ['affair', 0.006987916727325669], ['affection', 0.007036443926820986], ['afford', 0.007084971126316304], ['afor', 0.007133498325811621], ['afraid', 0.00771582

In [ ]:
models = []
for doc in docs:
  models.append(gen_unigram_model(doc))
  print(generate(5,models[-1]))


['taken', 'everi', 'and', 'wait', 'woman']
['and', 'rash', 'past', 'soothsay', 'and']
['you', 'stale', 'damn', 'death', 'thi']
['after', 'and', 'seueral', 'they', 'yong']
['would', 'could', 'desperado', 'employ', 'quest']
['bit', 'the', 'chimney', 'you', 'onli']
